# Exploration of Google Local Data (2021)

### Table of Contents

1. [Import Packages](#import)    
2. [Webscrape and Parse URL](#webscrape)   
3. [Display Tables of Interest](#table)   
4. [Extract Download Links](#extract)   
5. [Download and Explore Data](#download)   
6. [Summary](#summary)

**Objective:**

The main objective of this notebook is to explore the `Google Local Data (2021)` and extract important statistics for subsequent **Exploratory Data Analysis (EDA)**. 

**Description**

This Dataset contains review information on Google map (ratings, text, images, etc.), business metadata (address, geographical info, descriptions, category information, price, open hours, and MISC info), and links (relative businesses) up to Sep 2021 in the United States.

## 1. Import Packages<a class="anchor" id="import"></a>

In [1]:
import os
import requests
import sys

import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display, HTML

sys.path.append("../src/utils/eda")

from eda_utils import (
    download_file, parse_into_dataframe, remove_files_from_folder
)

## 2. Webscrape and Parse URL<a class="anchor" id="webscrape"></a>


In the notebook, we analyzed the `Google Local Data (2021)` available through the provided URL. Within this data, we identified three key tables of interest:

1. The first table is a summary table presenting essential statistics, including the total number of reviews, users, and businesses in the dataset. 
2. The second table contains comprehensive review data, organized by states, along with the corresponding review counts and metadata for each state. 
3. Lastly, the third table comprises a subset of review data, also organized by states, but this time featuring k-core reviews and ratings for each state. 

These tables serve as valuable sources of information for further analysis and insights into the Google Local Data.

**Retrieves and Parses contents of URL using `requests` and `BeautifulSoup4`:**

In [2]:
url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

**Locate all three tables of interest:**

In [3]:
summary_table = soup.find(
    lambda tag: tag.name=='table' and 
    tag.findChild('td').contents == ['Reviews:']
)
complete_data_table = soup.find(
    lambda tag: tag.name=='table' and 
    "reviews" in tag.findChildren('td')[1].get_text()
)
subset_data_table = soup.find(
    lambda tag: tag.name=='table' and 
    "10-core" in tag.findChildren('td')[1].get_text()
)

## 3. Display Tables of Interest<a class="anchor" id="table"></a>


### A. Summary Table

Here are the following summary statistics of `Google Local Data (2021)`:
- **666,324,103** reviews
- **113,643,107** users 
- **4,963,111** businesses

In [4]:
display(HTML(summary_table.prettify()))

Reviews:,"666,324,103"
Users:,"113,643,107"
Businesses:,"4,963,111"


### B. Complete Review Data Table

In [5]:
display(HTML(complete_data_table.prettify()))

Alabama,"reviews (8,967,499 reviews)","metadata (74,967 businesses)"
Alaska,"reviews (1,051,246 reviews)","metadata (12,774 businesses)"
Arizona,"reviews (18,375,050 reviews)","metadata (108,579 businesses)"
Arkansas,"reviews (5,106,056 reviews)","metadata (47,246 businesses)"
California,"reviews (70,529,977 reviews)","metadata (515,961 businesses)"
Colorado,"reviews (15,681,222 reviews)","metadata (106,829 businesses)"
Connecticut,"reviews (5,181,800 reviews)","metadata (49,200 businesses)"
Delaware,"reviews (1,885,948 reviews)","metadata (14,706 businesses)"
District of Columbia,"reviews (1,894,317 reviews)","metadata (11,060 businesses)"
Florida,"reviews (61,803,524 reviews)","metadata (378,020 businesses)"
Georgia,"reviews (24,060,125 reviews)","metadata (166,381 businesses)"


### C. Subset Review Data Table

In [6]:
display(HTML(subset_data_table.prettify()))

Alabama,"10-core (5,146,330 reviews)","ratings only (8,967,499 ratings)"
Alaska,"10-core (521,515 reviews)","ratings only (1,051,246 ratings)"
Arizona,"10-core (10,764,435 reviews)","ratings only (18,375,050 ratings)"
Arkansas,"10-core (2,855,468 reviews)","ratings only (5,106,056 ratings)"
California,"10-core (44,476,890 reviews)","ratings only (70,529,977 ratings)"
Colorado,"10-core (8,738,271 reviews)","ratings only (15,681,222 ratings)"
Connecticut,"10-core (2,680,107 reviews)","ratings only (5,181,800 ratings)"
Delaware,"10-core (905,537 reviews)","ratings only (1,885,948 ratings)"
District of Columbia,"10-core (564,783 reviews)","ratings only (1,894,317 ratings)"
Florida,"10-core (35,457,319 reviews)","ratings only (61,803,524 ratings)"
Georgia,"10-core (13,599,687 reviews)","ratings only (24,060,125 ratings)"


## 4. Extract Download Links<a class="anchor" id="extract"></a>

In this section, we will retrieve the "href" links from the Complete and Subset tables and store them in separate arrays. Specifically, we'll extract the URLs associated with the "href" attribute from anchor tags in both tables and save them in distinct arrays based on their respective tables.

**Complete Review Data Table:**

In [7]:
complete_table_rows = complete_data_table.findAll(lambda tag: tag.name=='td')
href_links = [
    row.find('a')['href'] for row in complete_table_rows if row.find('a')
]
complete_review_links = [link for link in href_links if "review" in link]
complete_meta_links = [link for link in href_links if "meta" in link]

**Subset Review Data Table:**

In [8]:
subset_table_rows = subset_data_table.findAll(lambda tag: tag.name=='td')
href_links = [
    row.find('a')['href'] for row in subset_table_rows if row.find('a')
]
subset_review_links = [link for link in href_links if "review" in link]
subset_rating_links = [link for link in href_links if "rating" in link]

**Print sample of download links:**

In [9]:
complete_review_links

['https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Alabama.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Alaska.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Arizona.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Arkansas.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Colorado.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Connecticut.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Delaware.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-District_of_Columbia.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Florida.json.gz',
 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-Georgi

## 5. Download and Explore Data<a class="anchor" id="download"></a>

In this part, we will download and investigate the initial file from each type of link to identify the necessary data for our project.

**Extract initial URLs:**

In [10]:
initial_links = [
    complete_review_links[0], 
    complete_meta_links[0], 
    subset_review_links[0], 
    subset_rating_links[0]
]

**Download the initial files using `download_file` utils function:**

In [11]:
gzip_folderpath = "../data/gzip"
for url in initial_links:
    download_file(url, gzip_folderpath)

INFO:root:Downloaded and saved to: ../data/gzip/review-Alabama.json.gz
INFO:root:Downloaded and saved to: ../data/gzip/meta-Alabama.json.gz
INFO:root:Downloaded and saved to: ../data/gzip/review-Alabama_10.json.gz
INFO:root:Downloaded and saved to: ../data/gzip/rating-Alabama.csv.gz


**Read downloaded data into DataFrame:**

`review_dataframe`: First complete review data (Alabama)   
`meta_dataframe`: First complete review metadata (Alabama)   
`subset_review_dataframe`: First subset review data (Alabama)   
`rating_dataframe`: First subset review ratings (Alabama)   

In [12]:
filepaths = [
    os.path.join(gzip_folderpath, link.split("/")[-1]) 
    for link in initial_links
]
review_dataframe, meta_dataframe, subset_review_dataframe, rating_dataframe = [
    parse_into_dataframe(filepath) for filepath in filepaths
]

**Columns of `review_dataframe`:**

`user_id`: A unique identifier for reviewer.   
`name`: The name of the reviewer.    
`time`: The unix timestamp of when the review was posted.    
`rating`: The rating given by the reviewer for the service or experience.    
`text`: The main content of the review.    
`pics`: A list of dictionaries, where each dictionary contains a URL to an image related to the review.    
`resp`: A dictionary containing the response from the business owner or management. It includes the timestamp of the response and the text of the response.   
`gmap_id`: The Google Maps ID associated with the place or location being reviewed.


<div style="background-color: black; color: white; padding: 10px">
    <p><b>Insight</b>: For our task, the columns of interest are <b>rating</b>, <b>text</b>, <b>resp</b>.</p>
</div>

In [13]:
review_dataframe

,user_id,name,time,rating,text,pics,resp,gmap_id
0,103378997989317552200,Hadley Coan,1626307876654,5.0,Love this little local shop. I shopped for my ...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,112597120686891028387,Macey Johnson,1626309762833,5.0,The absolute BEST children’s boutique in town!...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,100270366925303470748,Kelby Copeland,1626301530535,5.0,The absolute BEST place for children’s clothes...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,104473837835817861429,Wynn Thomas,1626300613475,5.0,The best children’s boutique in North Alabama ...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,114435642252064696269,Seth Meek,1624983585963,1.0,Terrible service. went in to buy 2 bogg bags a...,None,"{'time': 1624988771302, 'text': 'Hi Seth, my n...",0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
...,...,...,...,...,...,...,...,...
8967494,113553461507404622618,Tahniyath Sultana,1575316959599,5.0,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97
8967495,114905707698977221595,Cody Mc,1541393753107,5.0,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97
8967496,114609977215690498119,Liam Wood,1499539724361,5.0,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97
8967497,105458096176315452031,John Laine,1506555396467,3.0,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97


**Columns of `meta_dataframe`:**

`name`: The name of the business.     
`address`: The complete address of the place, including street, city, state, and postal code.     
`gmap_id`: The Google Maps ID associated with the business.    
`description`: The description of the business.   
`latitude`: The latitude coordinate of the business.    
`longitude`: The longitude coordinate of the business.    
`category`: The category of the business.    
`avg_rating`: The average rating of the business.    
`num_of_reviews`: The number of reviews for the business.    
`price`: The price category of the business.    
`hours`: The operating hours of the place, provided as a list of lists, where each inner list contains the day and the corresponding opening and closing time.    
`MISC`: A dictionary containing miscellaneous information about the place, such as service options, health & safety measures, accessibility options, planning details, and accepted payment methods.    
`state`: The current status of the business (e.g., permanently closed)    
`relative_results`: A list of Google Maps IDs related to this place.    
`url` The URL link to the Google Maps page for this place.    

<div style="background-color: black; color: white; padding: 10px">
    <p><b>Insight</b>: The metadata can serve as a good traceback to the main review data which can be matched using <b>gmap_id</b>.</p>
</div>

In [14]:
meta_dataframe

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Purple Peanut,"Purple Peanut, 2357 Whitesburg Dr # D, Huntsvi...",0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,None,34.713163,-86.574041,[Boutique],4.6,17,None,"[[Thursday, Closed], [Friday, 10AM–5PM], [Satu...","{'Service options': ['In-store shopping', 'Del...",Closed ⋅ Opens 10AM Fri,"[0x886269389730ce8b:0xedffba4037968914, 0x8862...",https://www.google.com/maps/place//data=!4m2!3...
1,Posh Mommy & Baby Too!,"Posh Mommy & Baby Too!, 7755 Hwy 72 W, Madison...",0x886268e8fdc4fd2f:0x746533eb9aa4d4df,"Stylish children's shop stocking apparel, mate...",34.753758,-86.739241,"[Baby store, Children's furniture store]",3.9,58,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...","{'Service options': ['In-store shopping', 'Del...",Open ⋅ Closes 5:30PM,"[0x88626ca5952a1829:0x263122b2d5a8e211, 0x8862...",https://www.google.com/maps/place//data=!4m2!3...
2,Soccer Shots North Alabama,None,0x8862693a967c0ecb:0xb84b19fa307337b0,None,34.651859,-86.765534,"[Soccer club, Service establishment]",4.7,3,None,None,None,None,"[0x88626ccc2e5fd0d5:0x3c2f2cb8d56688fd, 0x8862...",https://www.google.com/maps/place//data=!4m2!3...
3,Perdido Pass Jetty,"Perdido Pass Jetty, Orange Beach, AL 36561",0x889a07c7ba3993d5:0xbffff446052a43ed,None,30.273880,-87.559433,[Fishing pier],4.7,6,None,None,None,None,"[0x889a048d87bce635:0x3241501e0c4524a7, 0x8890...",https://www.google.com/maps/place//data=!4m2!3...
4,Tri-Rx Pharmaceuticals,"Tri-Rx Pharmaceuticals, 120 Vintage Dr NE, Hun...",0x8862151721b911fd:0xef80131fdd2f539b,None,34.791450,-86.532249,[Corporate office],3.7,6,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,None,https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74962,Aafes Xpress,"Aafes Xpress, 6600 Andrews Ave, Fort Rucker, A...",0x88926454984e0187:0x9ae0a1b39177b83a,None,31.323993,-85.723925,[Gas station],4.4,35,None,"[[Thursday, 5AM–11PM], [Friday, 5AM–12AM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,NaN,"[0x889264e06b427ea3:0x9e1b581a44d4dc8d, 0x8892...",https://www.google.com/maps/place//data=!4m2!3...
74963,Redstone Exchange,"Redstone Exchange, Huntsville, AL 35808",0x88626c454574ccdf:0xd68e2c9c6da64862,None,34.695399,-86.631584,[E-commerce service],4.4,218,None,"[[Thursday, 10AM–6PM], [Friday, 10AM–6PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,NaN,"[0x88626c453e8433e1:0x544837d0ec5f80b9, 0x8862...",https://www.google.com/maps/place//data=!4m2!3...
74964,Redstone Arsenal Commissary,"Redstone Arsenal Commissary, 3224 Acton Dr, Hu...",0x88626c453e8433e1:0x544837d0ec5f80b9,None,34.695967,-86.631386,"[Grocery store, ATM, Supermarket]",4.4,618,None,"[[Thursday, 9AM–7PM], [Friday, 9AM–7PM], [Satu...","{'Service options': ['In-store shopping', 'Del...",NaN,"[0x88626c454574ccdf:0xd68e2c9c6da64862, 0x8862...",https://www.google.com/maps/place//data=!4m2!3...
74965,Renaissance Mobile Riverview Plaza Hotel,"Renaissance Mobile Riverview Plaza Hotel, 64 S...",0x889a4fda2fa29c93:0x2a4edb18ef98ea7c,"Contemporary hotel offering bright rooms, 2 re...",30.690927,-88.039327,"[Hotel, Indoor lodging, Meeting planning servi...",4.5,1778,None,None,None,NaN,None,https://www.google.com/maps/place//data=!4m2!3...


**Columns of `subset_review_dataframe`:**

`user_id`: A unique identifier for reviewer.   
`name`: The name of the reviewer.    
`time`: The unix timestamp of when the review was posted.    
`rating`: The rating given by the reviewer for the service or experience.    
`text`: The main content of the review.    
`pics`: A list of dictionaries, where each dictionary contains a URL to an image related to the review.    
`resp`: A dictionary containing the response from the business owner or management. It includes the timestamp of the response and the text of the response.   
`gmap_id`: The Google Maps ID associated with the place or location being reviewed.


<div style="background-color: black; color: white; padding: 10px">
    <p><b>Insight</b>: For our task, the columns of interest are <b>rating</b>, <b>text</b>, <b>resp</b>.</p>
</div>

In [15]:
subset_review_dataframe

,user_id,name,time,rating,text,pics,resp,gmap_id
0,114043824230907811356,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,116009008420407071921,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,106239928178034609776,Sharon King,1547235290843,4,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,104970056115801744726,Veronica Pierce,1517709403534,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,110587471700525107383,Whitney Waldon Collier,1535245718492,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
...,...,...,...,...,...,...,...,...
5146325,113553461507404622618,Tahniyath Sultana,1575316959599,5,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97
5146326,114905707698977221595,Cody Mc,1541393753107,5,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97
5146327,114609977215690498119,Liam Wood,1499539724361,5,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97
5146328,105458096176315452031,John Laine,1506555396467,3,None,None,None,0x888912c75384e095:0x3bf8b383c85ccf97


**Columns of `rating_dataframe`:**

`business`, `user`, `rating`, `timestamp`

In [17]:
rating_dataframe

,business,user,rating,timestamp
0,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,103378997989317552200,5,1626307876654
1,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,112597120686891028387,5,1626309762833
2,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,100270366925303470748,5,1626301530535
3,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,104473837835817861429,5,1626300613475
4,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,114435642252064696269,1,1624983585963
...,...,...,...,...
8902586,0x888912c75384e095:0x3bf8b383c85ccf97,113553461507404622618,5,1575316959599
8902587,0x888912c75384e095:0x3bf8b383c85ccf97,114905707698977221595,5,1541393753107
8902588,0x888912c75384e095:0x3bf8b383c85ccf97,114609977215690498119,5,1499539724361
8902589,0x888912c75384e095:0x3bf8b383c85ccf97,105458096176315452031,3,1506555396467


**Remove initial files from `data/gzip` folder:**

In [18]:
remove_files_from_folder(gzip_folderpath)

NameError: name 'remove_files_from_folder' is not defined

## 6. Summary<a class="anchor" id="summary"></a>

This notebook downloads the gzip files and extracts into respective format. After initial exploration, we will proceed to perform a more extensive EDA (Exploratory Data Analysis).